# David Silver
* https://www.youtube.com/playlist?list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ
* http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching.html

## [Lecture 1](https://www.youtube.com/watch?v=2pWv7GOvuf0) [Intro](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/intro_RL.pdf)
* Each step, agent ...
  - execute action $A_t$
  - receives observation $O_t$ and reward $R_t$
* History $H_t\equiv\left\{O_t,R_t,A_t\right\}$
* *States*
  - envirnoment state $S_t^e=f(H_T)$ the information environment used to pick next observation/reward (independent of past history)
  - agent state $S_t^a=f(H_T)$ internal information used by agent to decide what to do next
* State is **Markov** iff independent of the past given present. Examples:
  - environment state $S_t^e$
  - full history $H_t$
* **Full observability**/**Markov decision process** $O_t=S_t^a=S_t^e$
* **Partial observability** $S_t^a\neq S_t^e$. Agent choose how to construct $S_t^a$, e.g.
  - $S_t^a=H_t$
  - Beliefs, i.e. a probability distribution of possible environment state
  - RNN, i.e. vector state via training $S_t^a=\sigma\left[{\rm Dense}\left(S_{t-1}^a,O_t\right)\right]$
* *Components of an agent*
  - **Policy**: action given state, either deterministic $a=\pi(s)$ or stochastic $\pi(a|s)$
  - **Value function**: prediction of future rewards (given the current policy) to evalute the favorability of a state: $v_\pi(s)=E\left[R_{t+1}+\gamma R_{t+2}+\gamma^2 R_{t+3}+...|S_t\right]$
  - **Model** (optional): predict the next state and next reward given current state and action
  ${\cal P}=p(S_{t+1}|S_t,A_t)$, ${\cal R}=p(R_{t+1}|S_t,A_t)$
* *Categorization of agent*
  * based on existence of value function and policy
    - **Value based**: has value function, on which policy is implied from
    - **Policy based**: has policy but no value function
    - **Actor critic**: has explicit value function and policy
  * based on existence of a model
    - **Model free**: based on policy/value function, without trying to figure out how the environment works
    - **Model based**
* Problems to be tackled:
    * **Learning** (unknown environment/rules, figure out policy through interaction) and **planning** (known env/rules, figure out best policy)
    * **Exploration** vs. **exploitation**
    * **Prediction** (evaluate future given policy) and **control** (find the best policy, i.e. need the prediction on all policies)

## [Lecture 2](https://www.youtube.com/watch?v=lfHX2hHRMVQ) [Markov Decision Processes](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/MDP.pdf)

* **Transition probability** ${\cal P_{ss'}}\equiv p(S_{t+1}=s'|S_t=s)=p(S_{t+1}|S_1...S_t)$ (2nd by definition)

* **Markov Process** $\left<{\cal S,P}\right>$ is specificed by the set of states ${\cal S}$ and transition probability ${\cal P}$

* **Markov Reward Process** $\left<{\cal S,P,R,\gamma}\right>$ is a markov process with (expected) rewards $R_s=\mathbb{E}(R_{t+1}|S_t=s)$ and discount factor $\gamma$
  - The goal (at each time step) is the maximize the return $G_t=\sum_{k=0}^\infty \gamma^k R_{t+k+1}$
  - $\gamma=0$ is myopic, $\gamma=1$ is far-sighted
  - Having discount factor means we have an imperfect model (also, mathematically convenient and avoids infinity if there're infinite loops)
  * **Value function of MRP** $v(s)=\mathbb{E}(G_t|S_t=s)$ is the expected total rewards (i.e. return) if we start off at a certain state
  * **Bellman Equation for MRP** decomposes value function's $G_t$ to immediate reward $R_{t+1}$ and the discounted value at the next step $v(s)=\mathbb{E}\left[R_{t+1}+\gamma v(S_{t+1})|S_t=s\right]$
    - use transition matrix to evalute the next state: $v(s)={\cal R}_s+\gamma \sum_{s'}{\cal P}_{ss'}v(s')$
    - cast everything to vectors/matrices, $v={\cal R}+\gamma{\cal P}v \implies v=\left(I-\gamma {\cal P}\right)^{-1}{\cal R}$
    - not practically for direct usage as O(n^3)
* **Markov Decision Process** $\left<{\cal S,A,P,R,\gamma}\right>$ adds actions $\cal A$ to a markov reward process
  - Transition matrix $\cal P_{ss'}^a$ depends on the action the agent take
  - Rewards $\cal R_s^a$ also depends on the action the agent take
  - **Policy** $\pi(a|s)=p(A_t=a|S_t=s)$ completely defines the behavior (actions) of the agent (only consider stationary policy i.e. independent of time)
  - Given policy which governs the action, $\left<{\cal S,P^\pi,R^\pi,\gamma}\right>$ reduces to a markov reward process, where $P^\pi=\sum_a \pi(a|s){\cal P}_{ss'}^a$
  - **State-value function of MDP** $v_\pi(s)=\mathbb{E}_\pi(G_t|S_t=s)$ is the expected return starting off at state $s$ and following policy $\pi$
  - **Action-value function of MDP** $q_\pi(s)=\mathbb{E}_\pi(G_t|S_t=s,A_t=a)$ is the expected return starting off at state $s$, *taking action $a$*, and following policy $\pi$
  - **Bellman Equation for MDP** still true: $v_\pi(s)=\mathbb{E}_\pi\left[R_{t+1}+\gamma v_\pi(S_{t+1})|S_t=s\right]$  and $q_\pi(s,a)=\mathbb{E}_\pi\left[R_{t+1}+\gamma q_\pi(S_{t+1},A_{t+1})|S_t=s,A_t=a\right]$
    - Condition on next action: $v_\pi(s)=\sum_a\pi(a|s)q_\pi(s,a)$
    - Condition on the state after the action about to take:  $q_\pi(s,a)={\cal R}_s^a+\gamma\sum_{s'}{\cal P}_{ss'}^a v_\pi(s')$
    - Combining for $v$: $v_\pi(s)=\sum_a \pi(a|s) \left({\cal R}_s^a+\gamma\sum_{s'}{\cal P}_{ss'}^a v_\pi(s')\right)$
    - Combining for $q$: $q_\pi(s,a)={\cal R}_s^a+\gamma\sum_{s'}{\cal P}_{ss'}^a \sum_{a'}\pi(a'|s')q_\pi(s',a')$
    - One way to solve is to cast to MRP and then use $v_\pi={\cal R^\pi}+\gamma{\cal P^\pi}v_\pi \implies v_\pi=\left(I-\gamma {\cal P^\pi}\right)^{-1}{\cal R^\pi}$
  - **Optimal state-value function** $v_*(s)=\max_\pi v_\pi(s)$<br/>
    **Optimal action-value function** $q_*(s,a)=\max_\pi q_\pi(s,a)$ (the goal is to solve for this -- directly informs the correct policy to follow)
* **Optimality** of policy in MDP
  - **Ordering** of policy is defined as $\pi\geq\pi'$ if $v_\pi(s)\geq v_{\pi'}(s) \forall s$
  - Theorem:
    - there is at least a deterministic optimal policy $\pi_*\geq\pi, \forall \pi$
    - if there are multiple optimal policies, they all achieve the same optimal $v$ and $q$: $v_{\pi_*}(s)=v_*(s)$ amd $q_{\pi_*}(s,a)=q_*(s,a)$
    - if $q_*$ is known, $\pi_*(a|s)$ is to follow ${\rm argmax}_a q_*(s,a)$
* **Bellman Optimality Equations**  in MDP
  - $v_*(s)=\max_a q_*(s,a)$
  - $q_*(s,a)={\cal R}_s^a+\gamma\sum_{s'}{\cal P}_{ss'}^a v_*(s')$ just average over the possible states that the environment state will go into
  - Combining for $v$...
  - Combining for $q$...
  - Nonlinear due to max function
  - No closed form in general, use iterative methods:  Value Iteration, Policy Iteration, Q-learning, Sarsa
* **Extensions**
  - Infinite (# states/action spaces) continuous (states/action spaces or time)
  - Partially observable MDP: add observations $\cal O$ and observation function $p(o|s,a)$ to problem definition
  

## [Lecture 3](https://www.youtube.com/watch?v=Nd1-UUMVfz4) [Planning by Dynamic Programming](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/DP.pdf)